In [ ]:
!pip install '../input/pytorch-190/torch-1.9.0+cu111-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '../input/pytorch-190/torchvision-0.10.0+cu111-cp37-cp37m-linux_x86_64.whl' --no-deps

In [ ]:
!pip install '/kaggle/input/mmdetection-v217/mmdetection/addict-2.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetection-v217/mmdetection/yapf-0.31.0-py2.py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetection-v217/mmdetection/terminal-0.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetection-v217/mmdetection/terminaltables-3.1.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetection-v217/mmdetection/mmcv_full-1.3.x-py2.py3-none-any/mmcv_full-1.3.16-cp37-cp37m-manylinux1_x86_64.whl' --no-deps
!cp -R ../input/mmdetection-v217/mmdetection/pycocotools-2.0.2 ./
!cp -R ../input/mmdetection-v217/mmdetection/mmpycocotools-12.0.3 ./
!pip install './pycocotools-2.0.2/pycocotools-2.0.2' --no-deps
!pip install './mmpycocotools-12.0.3/mmpycocotools-12.0.3' --no-deps

!rm -rf mmdetection

!cp -r /kaggle/input/mmdetection-v217/mmdetection/mmdetection-2.18.0 /kaggle/working/
!mv /kaggle/working/mmdetection-2.18.0 /kaggle/working/mmdetection
%cd /kaggle/working/mmdetection
!pip install -e .

# !rm -rf mmdetection

# !git clone https://github.com/open-mmlab/mmdetection.git /kaggle/working/mmdetection

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
#os.environ["PYTORCH_NO_CUDA_MEMORY_CACHING"] = str(1)
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import sklearn
import torchvision
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import cupy as cp
import shutil
import matplotlib.pyplot as plt
import PIL
import json
from PIL import Image, ImageEnhance
import albumentations as A
import mmdet
import mmcv
from albumentations.pytorch import ToTensorV2
import seaborn as sns
import glob
from pathlib import Path
import pycocotools
from pycocotools import mask
import numpy.random
import random
import cv2
import re
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.apis import inference_detector, init_detector, show_result_pyplot, set_random_seed

In [ ]:
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def flatten_l_o_l(nested_list):
    """ Flatten a list of lists """
    return [item for sublist in nested_list for item in sublist]


def load_json_to_dict(json_path):
    """ tbd """
    with open(json_path) as json_file:
        data = json.load(json_file)
    return data

In [ ]:
def get_img_and_mask(img_path, annotation, width, height):
    """ Capture the relevant image array as well as the image mask """
    img_mask = np.zeros((height, width), dtype=np.uint8)
    for i, annot in enumerate(annotation): 
        img_mask = np.where(rle_decode(annot, (height, width))!=0, i, img_mask)
    img = cv2.imread(img_path)[..., ::-1]
    return img[..., 0], img_mask

def plot_img_and_mask(img, mask, invert_img=True, boost_contrast=True):
    """ Function to take an image and the corresponding mask and plot
    
    Args:
        img (np.arr): 1 channel np arr representing the image of cellular structures
        mask (np.arr): 1 channel np arr representing the instance masks (incrementing by one)
        invert_img (bool, optional): Whether or not to invert the base image
        boost_contrast (bool, optional): Whether or not to boost contrast of the base image
        
    Returns:
        None; Plots the two arrays and overlays them to create a merged image
    """
    plt.figure(figsize=(20,10))
    
    plt.subplot(1,3,1)
    _img = np.tile(np.expand_dims(img, axis=-1), 3)
    
    # Flip black-->white ... white-->black
    if invert_img:
        _img = _img.max()-_img
        
    if boost_contrast:
        _img = np.asarray(ImageEnhance.Contrast(Image.fromarray(_img)).enhance(16))
        
    plt.imshow(_img)
    plt.axis(False)
    plt.title("Cell Image", fontweight="bold")
    
    plt.subplot(1,3,2)
    _mask = np.zeros_like(_img)
    _mask[..., 0] = mask
    plt.imshow(mask, cmap='rainbow')
    plt.axis(False)
    plt.title("Instance Segmentation Mask", fontweight="bold")
    
    merged = cv2.addWeighted(_img, 0.75, np.clip(_mask, 0, 1)*255, 0.25, 0.0,)
    plt.subplot(1,3,3)
    plt.imshow(merged)
    plt.axis(False)
    plt.title("Cell Image w/ Instance Segmentation Mask Overlay", fontweight="bold")
    
    plt.tight_layout()
    plt.show()

In [ ]:
def polygonFromMask(maskedArr, idx):
    # adapted from https://github.com/hazirbas/coco-json-converter/blob/master/generate_coco_json.py
    contours, _ = cv2.findContours(maskedArr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    segmentation = []
    valid_poly = 0
    for contour in contours:
        # Valid polygons have >= 6 coordinates (3 points)
         if contour.size >= 6:
            segmentation.append(contour.astype(float).flatten().tolist())
            valid_poly += 1
    if valid_poly == 0:
        raise ValueError(idx)
    return [segmentation]

In [ ]:
os.chdir("/kaggle/input/")

In [ ]:
train_df = pd.read_csv('../input/annotation-correction-v2/train.csv')

In [ ]:
lines = []
for f in train_df.itertuples():
    lines.append('../input/sartorius-cell-instance-segmentation/train/' + f[1] + '.png')

In [ ]:
lins = pd.Series(lines, name='img_path')

In [ ]:
train_df = pd.concat([train_df, lins], axis=1)

In [ ]:
tmp_df = train_df.drop_duplicates(subset=["id", "img_path"]).reset_index(drop=True)
tmp_df["annotation"] = train_df.groupby("id")["annotation"].agg(list).reset_index(drop=True)
train_df = tmp_df.copy()

In [ ]:
def plot_mask(idx):
    im, mk = get_img_and_mask(**train_df[["img_path", "annotation", "width", "height"]].iloc[idx].to_dict())
    plot_img_and_mask(im, mk)

In [ ]:
# plot_mask(0)

In [ ]:
train_df, val_df = train_test_split(train_df, train_size=0.95, random_state=0)

In [ ]:
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [ ]:
os.chdir("/kaggle/working/")

In [ ]:
%%writefile labels.txt
shsy5y
cort
astro

In [ ]:
os.chdir("/kaggle/working/mmdetection")
#!ls configs/detectors

In [ ]:
from mmcv import Config
# cfg = Config.fromfile('/kaggle/working/mmdetection/configs/htc/htc_x101_64x4d_fpn_dconv_c3-c5_mstrain_400_1400_16x1_20e_coco.py')
cfg = Config.fromfile('configs/detectors/detectors_htc_r50_1x_coco.py')
# cfg = Config.fromfile('/kaggle/working/mmdetection/configs/cascade_rcnn/cascade_mask_rcnn_r50_fpn_20e_coco.py')

In [ ]:
# print(cfg.pretty_text)

In [ ]:
cfg.dataset_type = 'CocoDataset'
cfg.classes = '/kaggle/working/labels.txt'
cfg.data_root = '/kaggle/working'
for head in cfg.model.roi_head.bbox_head:
    head.num_classes = 3

# cfg.model.roi_head.mask_head.semantic_head.num_classes=3
for head in cfg.model.roi_head.mask_head:
    head.num_classes=3
    
cfg.data.test.type = 'CocoDataset'
cfg.data.test.classes = 'labels.txt'
cfg.data.test.data_root = '/kaggle/working'
cfg.data.test.ann_file = '../input/my-json/val_dataset.json'
cfg.data.test.img_prefix = ''

cfg.data.train.type = 'CocoDataset'
cfg.data.train.data_root = '/kaggle/working'
cfg.data.train.ann_file = '../input/my-json/train_dataset.json'
cfg.data.train.seg_prefix=''

cfg.data.train.img_prefix = ''
cfg.data.train.classes = 'labels.txt'

cfg.data.val.type = 'CocoDataset'
cfg.data.val.data_root = '/kaggle/working'
cfg.data.val.ann_file = '../input/my-json/val_dataset.json'
cfg.data.val.img_prefix = ''
cfg.data.val.classes = 'labels.txt'

albu_train_transforms = []

#img_scale=[(1333, 800), (1056, 780)],

cfg.img_norm_cfg = dict(
   mean=[128, 128, 128], std=[11.58, 11.58, 11.58], to_rgb=True)
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='LoadAnnotations', with_bbox=True, with_mask=True, with_seg=True),
    dict(
        type='Resize',
        img_scale=[(1333, 800), (1056, 780)],
        #img_scale=[(1333, 400), (1333, 1200)],
        multiscale_mode='value',
        keep_ratio=True),
    dict(type='RandomFlip', direction=['horizontal', 'vertical'], flip_ratio=0.5),
    dict(type='Normalize', **cfg.img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='SegRescale', scale_factor=1 / 8),
    dict(type='DefaultFormatBundle'),
    dict(
        type='Collect',
        keys=['img', 'gt_bboxes', 'gt_labels', 'gt_masks', 'gt_semantic_seg']),
]

# cfg.val_pipeline = [
#     dict(type='LoadImageFromFile'),
#     dict(
#         type='MultiScaleFlipAug',
# #         img_scale=[(880, 1192), (960, 130), (1040, 1408), (1160, 1570), (1240, 1678)],
#         img_scale = [(1333, 800), (1690, 960)],
#         flip=False,
#         transforms=[
#         dict(type='Collect', keys=['img'])
#         ])
# ]

#img_scale=[(1333, 800), (1056, 780)],
cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',

        img_scale=[(1333, 800), (1056, 780)],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            #dict(type='RandomFlip',direction=['horizontal', 'vertical'],
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]


cfg.data.train.pipeline = cfg.train_pipeline
cfg.data.val.pipeline = cfg.test_pipeline
cfg.data.test.pipeline = cfg.test_pipeline

cfg.model.test_cfg.rcnn.max_per_img = 380

cfg.load_from = '../input/checkpointhope/detectors_htc_r50_1x_coco-329b1453.pth'

cfg.work_dir = '/kaggle/working/model_output'

cfg.optimizer.lr = 0.004
cfg.data.samples_per_gpu = 1
cfg.data.workers_per_gpu = 0

cfg.lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=500,
    warmup_ratio=0.001,
    step=[8, 11])

# cfg.lr_config = dict(
#     policy='CosineAnnealing', 
#     by_epoch=False,
#     warmup='linear', 
#     warmup_iters=125, 
#     warmup_ratio=0.001,
#     min_lr=1e-07)

cfg.evaluation.metric = 'segm'
cfg.roi_layer=dict(type='RoIAlign', output_size=7, sampling_ratio=0,use_torchvision=True)
cfg.model.roi_head.bbox_roi_extractor.roi_layer=dict(type= 'RoIAlign', output_size= 7, sampling_ratio= 0,use_torchvision=True)
cfg.model.roi_head.mask_roi_extractor.roi_layer=dict(type= 'RoIAlign', output_size= 14, sampling_ratio= 0,use_torchvision=True)
cfg.model.roi_head.mask_roi_extractor.roi_layer=dict(type= 'RoIAlign', output_size= 14, sampling_ratio= 0,use_torchvision=True)

cfg.checkpoint_config = dict(interval=1,max_keep_ckpts=8)
cfg.runner = dict(type='EpochBasedRunner', max_epochs=15)
cfg.seed=22
cfg.gpu_ids = range(1)
#cfg.fp16 = dict(loss_scale=512.0)
meta = dict()
meta['config'] = cfg.pretty_text
#print(f'Config:\n{cfg.pretty_text}')

In [ ]:
os.chdir("/kaggle/working")

In [ ]:
datasets = [build_dataset(cfg.data.train)]

In [ ]:
datasets

In [ ]:
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
model.CLASSES = datasets[0].CLASSES
mmcv.mkdir_or_exist(os.path.abspath(cfg.work_dir))

In [ ]:
train_detector(model, datasets, cfg, distributed=False, validate=False)

In [ ]:
# confidence_thresholds = {0: 0.25, 1: 0.55, 2: 0.35}

In [ ]:
# def get_mask_from_result(result):
#     d = {True : 1, False : 0}
#     u,inv = np.unique(result,return_inverse = True)
#     mk = cp.array([d[x] for x in u])[inv].reshape(result.shape)
# #     print(mk.shape)
#     return mk

# def does_overlap(mask, other_masks):
#     for other_mask in other_masks:
#         if np.sum(np.logical_and(mask, other_mask)) > 0:
#             return True
#     return False


# def remove_overlapping_pixels(mask, other_masks):
#     for other_mask in other_masks:
#         if np.sum(np.logical_and(mask, other_mask)) > 0:
#             #print("Overlap detected")
#             mask[np.logical_and(mask, other_mask)] = 0
#     return mask

# def rle_encoding(x):
#     dots = np.where(x.flatten() == 1)[0]
#     run_lengths = []
#     prev = -2
#     for b in dots:
#         if (b>prev+1): run_lengths.extend((b + 1, 0))
#         run_lengths[-1] += 1
#         prev = b
#     return ' '.join(map(str, run_lengths))

# from skimage import measure
# from skimage.filters import unsharp_mask
# def unsharp(img):
#     unsharp_img = unsharp_mask(img, radius=5, amount=1.0)

#     return unsharp_img

In [ ]:
# import os
# model_output = os.path.join('../input/mmdetection-new-cfg-training', 'model_output')
# os.listdir(model_output)

In [ ]:
# model = init_detector(cfg, '../input/mmdetection-new-cfg-training/model_output/epoch_15.pth')

In [ ]:
# segms = []
# files = []
# for file in sorted(os.listdir('../input/sartorius-cell-instance-segmentation/test')):
#     img = mmcv.imread('../input/sartorius-cell-instance-segmentation/test/' + file)
#     #img = unsharp(img)
# #     plt.imshow(img)
# #     plt.show()
# #     img = (unsharp(img)*255)
# #     plt.imshow(img)
#     result = inference_detector(model, img)
#     #print(result[0])
#     show_result_pyplot(model, img, result)
#     previous_masks = []
#     for i, classe in enumerate(result[0]):
# #         print(classe)
#         if classe.shape != (0, 5):
#             bbs = classe
#             sgs = result[1][i]
#             for bb, sg in zip(bbs,sgs):
#                 box = bb[:4]
#                 cnf = bb[4]
#                 if cnf >= confidence_thresholds[i]:
#                     mask = get_mask_from_result(sg)
#                     mask = remove_overlapping_pixels(mask, previous_masks)
#                     previous_masks.append(mask)

#     for mk in previous_masks:
#             rle_mask = rle_encoding(mk)
#             segms.append(rle_mask)
#             files.append(str(file.split('.')[0]))

In [ ]:
# indexes = []
# for i, segm in enumerate(segms):
#     if segm == '':
#         indexes.append(i)

In [ ]:
# for element in sorted(indexes, reverse = True):
#     del segms[element]
#     del files[element]

In [ ]:
# files = pd.Series(files, name='id')
# preds = pd.Series(segms, name='predicted')

In [ ]:
# submission_df = pd.concat([files, preds], axis=1)

In [ ]:
# shutil.rmtree('/kaggle/working/mmpycocotools-12.0.3')

In [ ]:
# shutil.rmtree('/kaggle/working/mmdetection')

In [ ]:
# shutil.rmtree('/kaggle/working/pycocotools-2.0.2')

In [ ]:
# shutil.rmtree('/kaggle/working/model_output')

In [ ]:
# os.remove('/kaggle/working/labels.txt')

In [ ]:
# submission_df.to_csv('submission.csv', index=False)

In [ ]:
# submission_df